In [ ]:
from pynq import Overlay
from pynq import MMIO
import time

ol = Overlay('top_wrapper.xsa')
ol.ip_dict.keys()

In [ ]:
ol.ip_dict['axi_quad_spi_0']

In [ ]:
# 初始化 AXI Quad SPI 的 base address（從 Vivado Address Editor 抓）
spi = MMIO(0x41E00000, 0x10000)

# SPI 寄存器 offset 定義（根據 PG153）
SPICR  = 0x60  # 控制暫存器
SPISR  = 0x64  # 狀態暫存器
SPIDTR = 0x68  # 傳送資料暫存器
SPIDRR = 0x6C  # 接收資料暫存器
SPISSR = 0x70  # Slave Select 設定暫存器
SPISRR = 0x40  # 軟體重置暫存器

# -------------------------------
# Step 1️⃣ Reset SPI core
# -------------------------------
spi.write(SPISRR, 0x0A)  # 根據 PG153，寫入 0x0A 觸發 reset

# -------------------------------
# Step 2️⃣ SPI 初始化：啟用 Master 模式 & Manual SS 模式
# -------------------------------
# Bit 1 = SPE (SPI Enable)
# Bit 2 = Master mode
# Bit 7 = Manual Slave Select (使用 SPISSR 控制 /CS)
spi.write(SPICR, (1 << 2) | (1 << 1) | (1 << 7))  # = 0b10000110 = 0x86

# -------------------------------
# Step 3️⃣ 啟用 slave：拉低 /CS（CS 為 active-low）
# -------------------------------
spi.write(SPISSR, 0x0)  # 選擇 slave 0（拉低片選）

# -------------------------------
# Step 4️⃣ 清空 RX FIFO（避免先前雜訊）
# -------------------------------
while not (spi.read(SPISR) & 0x01):  # Bit 0 = RX_Empty，0 表 FIFO 有資料
    spi.read(SPIDRR)  # 丟棄先前資料

# -------------------------------
# Step 5️⃣ 發送 JEDEC ID 指令（0x9F）
# -------------------------------
spi.write(SPIDTR, 0x9F)  # 發送 0x9F，要求 JEDEC ID
while spi.read(SPISR) & 0x01: pass  # 等待接收完成
spi.read(SPIDRR)  # 丟掉回傳的 0x9F 應答（無意義）

# -------------------------------
# Step 6️⃣ 發送 dummy byte 觸發接收 JEDEC 回應
# -------------------------------
jedec_id = []
for _ in range(3):  # JEDEC ID 通常為 3 bytes
    spi.write(SPIDTR, 0x00)  # 發送 dummy byte
    while spi.read(SPISR) & 0x01:  # 等待接收有效資料
        pass
    data = spi.read(SPIDRR) & 0xFF  # 只取 8-bit
    jedec_id.append(data)

# -------------------------------
# Step 7️⃣ 顯示結果
# -------------------------------
print("JEDEC ID:", jedec_id)

# ✅ 你讀到的 [0xEF, 0x40, 0x18] 解釋如下：
# Byte 十進位 十六進位 說明
# 1    239    0xEF    Winbond 廠商 ID
# 2    64     0x40    SPI NOR Flash
# 3    24     0x18    128Mbit = 16MB 容量
